# Getting NBA Linup Data From Rotowire.com

In [113]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [114]:
today = datetime.now().strftime('%Y%m%d')

In [2]:
url = "https://www.rotowire.com/basketball/nba-lineups.php"

In [4]:
response = requests.get(url)

In [117]:
type(response.status_code)

int

In [7]:
soup = BeautifulSoup(response.text, "html.parser")

In [134]:
soup

<!DOCTYPE html>
<html lang="en">
<head>
<script src="https://bet.rotowire.com/js/app.js"></script>
<script>
        window.dataLayer = window.dataLayer || [];
        window.dataLayer.push({"sport_name":"nba","sport_content":"fantasy","event":"content_grouping"});
    </script>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
    new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
    j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
    'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-WM527Q7');</script>
<!-- End Google Tag Manager -->
<script defer="" src="https://zz.connextra.com/dcs/tagController/tag/b13e35197587/homepage"></script>
<link href="/favicon.ico" rel="shortcut icon"/>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1, shrin

In [135]:
game_box_tag_type, game_box_div_class = "div", "lineup is-nba"
away_team_tag_type, away_team_tag_class, away_team_tag_id = "a", "lineup__team is-visit","img alt"
home_team_tag_type, home_team_tag_class, home_team_tag_id = "a", "lineup__team is-home", "img alt"
lineup_type_tag_type, lineup_type_tag_class, lineup_type_tag_id = "li", "lineup_status", "text"
away_team_lineup_tag_type, away_team_lineup_class = "ul", "lineup__list is-visit"
home_team_lineup_tag_type, home_team_lineup_ul_class = "ul", "lineup__list is-home"
lineup_player_tag_type, lineup_player_tag_class, lineup_player_status_tag_id, lineup_player_name = "ul", "lineup__player", "title", "text"

In [136]:
games = soup.find_all(game_box_tag_type, class_= game_box_div_class)

In [137]:
len(games)

7

In [79]:
nba_games = {}
games_list = []

In [ ]:
for game in games:

    game_dict = {}

    starter_list = []
    injury_list = []

    away_team_section = game.find(away_team_tag_type, class_= away_team_tag_class)
    if away_team_section:
        away_team_abbr = away_team_section.find("div", class_="lineup__abbr").text
    
    game_dict['away_team_abbreviated'] = away_team_abbr

    away_team_list_section = game.find("ul", class_ = "lineup__list is-visit")

    #print("Away Team Section", away_team_list_section)
    if away_team_list_section:

        list_items = away_team_list_section.find_all("li")
        
        for item in list_items:
            if item.get("class"):
                #print(item.get("class")[0])

                if item.get("class")[0] != "lineup__player":
                    #print(item.text)
                    lineup_type = item.text.strip()

                elif item.get("class")[0] == "lineup__player":
                    #print(lineup_type)
                    #print(item)
                    player_status = item.get("title")

                    player_position = item.find("div", class_="lineup__pos").get_text(strip = True)

                    player_name = item.find("a").get("title")

                    player_status_abbr_element = item.find("span", class_ = "lineup__inj")

                    if player_status_abbr_element:
                        player_status_abbr = player_status_abbr_element.text
                    
                    else:
                        player_status_abbr = "healthy"

                    if lineup_type == "Expected Lineup":
                        starter_list.append({"name":player_name,
                                             "position": player_position,
                                             "status": player_status,
                                             "status_abbr": player_status_abbr})
            
                    elif lineup_type == "MAY NOT PLAY":
                        injury_list.append({"name":player_name,
                                             "position": player_position,
                                             "status": player_status,
                                             "status_abbr": player_status_abbr})

        game_dict["away_team_projected_starters"] = starter_list

        game_dict["away_team_injury_list"] = injury_list

    starter_list = []
    injury_list = []


    home_team_section = game.find(home_team_tag_type, class_ = home_team_tag_class)
    if home_team_section:
        home_team_abbr = home_team_section.find("div", class_="lineup__abbr").text

    game_dict['home_team_abbreviated'] = home_team_abbr

    home_team_list_section = game.find("ul", class_ = "lineup__list is-home")

    #print("Away Team Section", away_team_list_section)
    if home_team_list_section:

        list_items = home_team_list_section.find_all("li")



        
        for item in list_items:
            if item.get("class"):
                #print(item.get("class")[0])

                if item.get("class")[0] != "lineup__player":
                    #print(item.text)
                    lineup_type = item.text.strip()

                elif item.get("class")[0] == "lineup__player":
                    #print(lineup_type)
                    player_status = item.get("title")

                    player_position = item.find("div", class_="lineup__pos").get_text(strip = True)

                    player_name = item.find("a").get("title")

                    player_status_abbr_element = item.find("span", class_ = "lineup__inj")

                    if player_status_abbr_element:
                        player_status_abbr = player_status_abbr_element.text
                    
                    else:
                        player_status_abbr = "healthy"

                    if lineup_type == "Expected Lineup":
                        starter_list.append({"name":player_name,
                                             "position": player_position,
                                             "status": player_status,
                                             "status_abbr": player_status_abbr})
            
                    elif lineup_type == "MAY NOT PLAY":
                        injury_list.append({"name":player_name,
                                             "position": player_position,
                                             "status": player_status,
                                             "status_abbr": player_status_abbr})

        game_dict["home_team_projected_starters"] = starter_list

        game_dict["home_team_injury_list"] = injury_list

    games_list.append(game_dict)
        


In [181]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

todaysdate = datetime.now().strftime('%Y%m%d')

todaysminute = datetime.now().strftime('%Y%m%d%H%M')

def get_nba_lineups():
    try:
        #setting the url to see the projected nba lineups and the injury list for today's nba matchups from rotwire.com
        url = "https://www.rotowire.com/basketball/nba-lineups.php"

        #sending a get request to the URL to receive the webpage data
        response = requests.get(url)

        #checking to see if the response status code shows successful before proceeding with the script to extract information
        if response.status_code == 200:
            print("Successfully received NBA Lineups from Rotowire")
            return response.text


        #if the status code from the API request was unsuccessful then we will print this message and the response.status_code
        else:
            print(f"Status code from rotowire request does not show a successful request. Status Code: {response.status_code}")
    #an exception block for if our function fails
    except Exception as e:
        print(f"An error occured trying to call the rotowire webpage with the injured player content: {e}")

def get_beautiful_soup(rotowire_content):
    try:
        soup = BeautifulSoup(rotowire_content)
        return soup
    except Exception as e:
        print(f"An Error Occurred trying to use the Beautiful Soup package on the response text. {e}")

def list_of_games(soup):
    try:
        soup = BeautifulSoup(response.text, "html.parser")

        #setting the kind of tag we are looking for and the html class we are looking for to get individual game data
        game_box_tag_type, game_box_div_class = "div", "lineup is-nba"

        #using the beautiful soup find_all function to get each chunk of game data into a list to iterate through
        game_list = soup.find_all(game_box_tag_type, class_= game_box_div_class)
        return game_list
    
    except Exception as e:
        print(f"An error occured trying to identify the different games listed on the webpage. {e}")

def home_team_abbreviation(game_content):
    try:
        home_team_tag_type, home_team_tag_class = "a", "lineup__team is-home"

        home_team_abbr_tag, home_team_abbr_class = "div", "lineup__abbr"

        #getting the chunk of html code that is limited to the away teams information in the scorebox
        home_team_section = game.find(home_team_tag_type, class_= home_team_tag_class)

        #setting a conditional to make sure that the away team section did not return None
        if home_team_section:
            
            #finding the away team's abbreviation by using the html tag and class name variables above, and calling out the text value after soup locates the HTML snippet
            home_team_abbr = home_team_section.find(home_team_abbr_tag, class_= home_team_abbr_class).text

            return home_team_abbr
        
    except Exception as E:
        print(f"An Error occurred while trying to get the away team abbreviation: {e}")

def away_team_abbreviation(game_content):
    try:
        away_team_tag_type, away_team_tag_class = "a", "lineup__team is-visit"

        away_team_abbr_tag, away_team_abbr_class = "div", "lineup__abbr"

        #getting the chunk of html code that is limited to the away teams information in the scorebox
        away_team_section = game.find(away_team_tag_type, class_= away_team_tag_class)

        #setting a conditional to make sure that the away team section did not return None
        if away_team_section:
            
            #finding the away team's abbreviation by using the html tag and class name variables above, and calling out the text value after soup locates the HTML snippet
            away_team_abbr = away_team_section.find(away_team_abbr_tag, class_= away_team_abbr_class).text

            return away_team_abbr
        
    except Exception as E:
        print(f"An Error occurred while trying to get the away team abbreviation: {e}")

def home_team_starters(game_content):
    try:

        player_list_html_tag, home_team_player_list_class = "ul", "lineup__list is-home"
        
        home_team_list_section = game.find(player_list_html_tag, class_ = home_team_player_list_class)
        
        player_listing_html_tag = "li"

        player_list_player_class = "lineup__player"

        starter_list = []

        #print("Away Team Section", away_team_list_section)
        if home_team_list_section:
                
            #getting all of the list items in the unordered list
            list_items = home_team_list_section.find_all(player_listing_html_tag)
            
            #iterating through each list item to get player data
            for item in list_items:
                #print(item)

                #seting a variable for the classes of an item (it will be a list of items)
                item_class = item.get("class")

                #print(item_class)
                #making sure the item_class list isn't empty
                if item_class:
                    #there are three types of classes starters, injured, and player. We need to identify if we have starters or injured players, and then parse the player information
                    if item_class[0].strip() == 'lineup__status':
                        lineup_status = item_class[1]

                    elif item_class[0] == player_list_player_class:

                        #finding the title of the item to get the long-form status of the player
                        player_status = item.get("title")

                        #finding the div and class holding the player position information and retrieving the text information for it
                        player_position = item.find("div", class_="lineup__pos").get_text(strip = True)

                        #finding the html element and getting the title of the element to get the players name
                        player_name = item.find("a").get("title")

                        #finding the span section which lists the abbreviated injury annotation
                        player_status_abbr_element = item.find("span", class_ = "lineup__inj")

                        #checking to see if the span returns a value or if it's empty
                        if player_status_abbr_element:

                            #if it returns a value we set the abbreviation to the text in the span
                            player_status_abbr = player_status_abbr_element.text
                        
                        #if it returns none we set the abbreviated status as healthy
                        else:
                            player_status_abbr = "healthy"

                        #appending starting player information to the starter list
                        starter_list.append({"name":player_name,
                                            "position": player_position,
                                            "status": player_status,
                                            "status_abbr": player_status_abbr}) 
                
                    else:
                        break
                
                    
            return starter_list, lineup_status

    except Exception as e:
        print(f"There was an error reading in the away team starter data: {e}")

def home_team_injuries(game_content):
    try:

        player_list_html_tag, home_team_player_list_class = "ul", "lineup__list is-home"
        
        home_team_list_section = game.find(player_list_html_tag, class_ = home_team_player_list_class)
        
        player_listing_html_tag = "li"

        player_list_player_class = "lineup__player"

        injury_list = []

        #print("Away Team Section", away_team_list_section)
        if home_team_list_section:
                
            #getting all of the list items in the unordered list
            list_items = home_team_list_section.find_all(player_listing_html_tag)
            
            #iterating through each list item to get player data
            start = None
            
            for item in list_items:    
                #print(item)

                #seting a variable for the classes of an item (it will be a list of items)
                item_class = item.get("class")

                
                #making sure the item_class list isn't empty
                if item_class:
                    #print(item_class[0], start)
                    #there are three types of classes starters, injured, and player. We need to identify if we have starters or injured players, and then parse the player information
                    if item_class[0].strip() == 'lineup__status':
                        continue

                    elif item_class[0].strip() == 'lineup__title':
                        start = "start"

                    elif item_class[0] == player_list_player_class and start:

                        #finding the title of the item to get the long-form status of the player
                        player_status = item.get("title")

                        #finding the div and class holding the player position information and retrieving the text information for it
                        player_position = item.find("div", class_="lineup__pos").get_text(strip = True)

                        #finding the html element and getting the title of the element to get the players name
                        player_name = item.find("a").get("title")

                        #finding the span section which lists the abbreviated injury annotation
                        player_status_abbr_element = item.find("span", class_ = "lineup__inj")

                        #checking to see if the span returns a value or if it's empty
                        if player_status_abbr_element:

                            #if it returns a value we set the abbreviation to the text in the span
                            player_status_abbr = player_status_abbr_element.text
                        
                        #if it returns none we set the abbreviated status as healthy
                        else:
                            player_status_abbr = "healthy"

                        #appending starting player information to the starter list
                        injury_list.append({"name":player_name,
                                            "position": player_position,
                                            "status": player_status,
                                            "status_abbr": player_status_abbr}) 
                    
                    else:
                        continue
                
                    
            return injury_list

    except Exception as e:
        print(f"There was an error reading in the home team injury data: {e}")

def away_team_starters(game_content):
    try:

        player_list_html_tag, away_team_player_list_class = "ul", "lineup__list is-visit"
        
        away_team_list_section = game.find(player_list_html_tag, class_ = away_team_player_list_class)
        
        player_listing_html_tag = "li"

        player_list_player_class = "lineup__player"

        starter_list = []

        #print("Away Team Section", away_team_list_section)
        if away_team_list_section:
                
            #getting all of the list items in the unordered list
            list_items = away_team_list_section.find_all(player_listing_html_tag)
            
            #iterating through each list item to get player data
            for item in list_items:
                #print(item)

                #seting a variable for the classes of an item (it will be a list of items)
                item_class = item.get("class")

                #print(item_class)
                #making sure the item_class list isn't empty
                if item_class:
                    #there are three types of classes starters, injured, and player. We need to identify if we have starters or injured players, and then parse the player information
                    if item_class[0].strip() == 'lineup__status':
                        lineup_status = item_class[1]

                    elif item_class[0] == player_list_player_class:

                        #finding the title of the item to get the long-form status of the player
                        player_status = item.get("title")

                        #finding the div and class holding the player position information and retrieving the text information for it
                        player_position = item.find("div", class_="lineup__pos").get_text(strip = True)

                        #finding the html element and getting the title of the element to get the players name
                        player_name = item.find("a").get("title")

                        #finding the span section which lists the abbreviated injury annotation
                        player_status_abbr_element = item.find("span", class_ = "lineup__inj")

                        #checking to see if the span returns a value or if it's empty
                        if player_status_abbr_element:

                            #if it returns a value we set the abbreviation to the text in the span
                            player_status_abbr = player_status_abbr_element.text
                        
                        #if it returns none we set the abbreviated status as healthy
                        else:
                            player_status_abbr = "healthy"

                        #appending starting player information to the starter list
                        starter_list.append({"name":player_name,
                                            "position": player_position,
                                            "status": player_status,
                                            "status_abbr": player_status_abbr}) 
                
                    else:
                        break
                
                    
            return starter_list, lineup_status

    except Exception as e:
        print(f"There was an error reading in the away team starter data: {e}")

def away_team_injuries(game_content):
    try:

        player_list_html_tag, away_team_player_list_class = "ul", "lineup__list is-visit"
        
        away_team_list_section = game.find(player_list_html_tag, class_ = away_team_player_list_class)
        
        player_listing_html_tag = "li"

        player_list_player_class = "lineup__player"

        injury_list = []

        #print("Away Team Section", away_team_list_section)
        if away_team_list_section:
                
            #getting all of the list items in the unordered list
            list_items = away_team_list_section.find_all(player_listing_html_tag)
            
            #iterating through each list item to get player data
            start = None
            
            for item in list_items:    
                #print(item)

                #seting a variable for the classes of an item (it will be a list of items)
                item_class = item.get("class")

                
                #making sure the item_class list isn't empty
                if item_class:
                    #print(item_class[0], start)
                    #there are three types of classes starters, injured, and player. We need to identify if we have starters or injured players, and then parse the player information
                    if item_class[0].strip() == 'lineup__status':
                        continue

                    elif item_class[0].strip() == 'lineup__title':
                        start = "start"

                    elif item_class[0] == player_list_player_class and start:

                        #finding the title of the item to get the long-form status of the player
                        player_status = item.get("title")

                        #finding the div and class holding the player position information and retrieving the text information for it
                        player_position = item.find("div", class_="lineup__pos").get_text(strip = True)

                        #finding the html element and getting the title of the element to get the players name
                        player_name = item.find("a").get("title")

                        #finding the span section which lists the abbreviated injury annotation
                        player_status_abbr_element = item.find("span", class_ = "lineup__inj")

                        #checking to see if the span returns a value or if it's empty
                        if player_status_abbr_element:

                            #if it returns a value we set the abbreviation to the text in the span
                            player_status_abbr = player_status_abbr_element.text
                        
                        #if it returns none we set the abbreviated status as healthy
                        else:
                            player_status_abbr = "healthy"

                        #appending starting player information to the starter list
                        injury_list.append({"name":player_name,
                                            "position": player_position,
                                            "status": player_status,
                                            "status_abbr": player_status_abbr}) 
                    
                    else:
                        continue
                
                    
            return injury_list

    except Exception as e:
        print(f"There was an error reading in the away team injury data: {e}")

def parse_nba_lineups(rotowire_content):
    #trying to call our function
    try:
            
        soup = BeautifulSoup(response.text, "html.parser")

        #setting the kind of tag we are looking for and the html class we are looking for to get individual game data
        game_box_tag_type, game_box_div_class = "div", "lineup__box"

        #using the beautiful soup find_all function to get each chunk of game data into a list to iterate through
        games = soup.find_all(game_box_tag_type, class_= game_box_div_class)

        #creating an empty dictionary to store my list of json's into for exporting to a document database
        nba_games = {}

        #setting an empty list to store game information into which will then be used to tie to today's date as the key for the nba_games dictionary
        games_list = []
        
        #setting variables to the html tag type and class type to find the away team information needed
        away_team_tag_type, away_team_tag_class = "a", "lineup__team is-visit"

        #setting the variables for the html tag and class type's to get the away team's abbreviation out of the score bug
        away_team_abbr_tag, away_team_abbr_class = "div", "lineup__abbr"

        #setting the variable for the html tag and class type to get the away team's list of starting and injured players
        player_list_html_tag, away_team_player_list_class, home_team_player_list_class = "ul", "lineup__list is-visit", "lineup__list is-home"

        player_listing_html_tag = "li"

        player_list_player_class = "lineup__player"

        #iterating through each gamebox's div section of html code
        for game in games:
            
            #setting an empty dictionary to store specific game data into, this will later be the dictionary we add into the games_list list.
            game_dict = {}

            #creating an empty list to hold the starting player information for a team
            starter_list = []

            #creating an empty list to hold the injured player information for a team
            injury_list = []
            
            #getting the chunk of html code that is limited to the away teams information in the scorebox
            away_team_section = game.find(away_team_tag_type, class_= away_team_tag_class)

            #setting a conditional to make sure that the away team section did not return None
            if away_team_section:
                
                #finding the away team's abbreviation by using the html tag and class name variables above, and calling out the text value after soup locates the HTML snippet
                away_team_abbr = away_team_section.find(away_team_abbr_tag, class_= away_team_abbr_class).text
            
            #adding the away team's abbreviated name into the game_dict dictionary
            game_dict['away_team_abbreviated'] = away_team_abbr

            #each team's player information is stored in an unordered list, this list holds both the starting players and injured players.
            #we need to use a for loop to run through each list item for the away team and use a conditional to determine if we are listing out starters or injured players
            #some starters can be listed as injured, and non-starters also appear on the injured list

            away_team_list_section = game.find(player_list_html_tag, class_ = away_team_player_list_class)
            
            #print("Away Team Section", away_team_list_section)
            if away_team_list_section:
                
                #getting all of the list items in the unordered list
                list_items = away_team_list_section.find_all(player_listing_html_tag)
                
                #iterating through each list item to get player data
                for item in list_items:
                    #seting a variable for the classes of an item (it will be a list of items)
                    item_class = item.get("class")

                    #making sure the item_class list isn't empty
                    if item_class:
                        
                        #there are three types of classes starters, injured, and player. We need to identify if we have starters or injured players, and then parse the player information
                        if item_class[0] != player_list_player_class:
                            
                            #setting the lineup type to know whether it is a starting lineup section or injured player section
                            lineup_type = item.text.strip()

                        elif item_class[0] == player_list_player_class:

                            #finding the title of the item to get the long-form status of the player
                            player_status = item.get("title")

                            #finding the div and class holding the player position information and retrieving the text information for it
                            player_position = item.find("div", class_="lineup__pos").get_text(strip = True)

                            #finding the html element and getting the title of the element to get the players name
                            player_name = item.find("a").get("title")

                            #finding the span section which lists the abbreviated injury annotation
                            player_status_abbr_element = item.find("span", class_ = "lineup__inj")

                            #checking to see if the span returns a value or if it's empty
                            if player_status_abbr_element:

                                #if it returns a value we set the abbreviation to the text in the span
                                player_status_abbr = player_status_abbr_element.text
                            
                            #if it returns none we set the abbreviated status as healthy
                            else:
                                player_status_abbr = "healthy"

                            #seeing if the lineup_type is expected lineup or injured list
                            if lineup_type == "Expected Lineup":

                                #appending starting player information to the starter list
                                starter_list.append({"name":player_name,
                                                    "position": player_position,
                                                    "status": player_status,
                                                    "status_abbr": player_status_abbr})

                            #appending injured player information to the injured player list
                            elif lineup_type == "MAY NOT PLAY":
                                injury_list.append({"name":player_name,
                                                    "position": player_position,
                                                    "status": player_status,
                                                    "status_abbr": player_status_abbr})                         
                        


            

    
    #an exception block for if our function fails
    except Exception as e:
        print(f"Something went wrong: {e}")

In [123]:
roto_lineups_html = get_nba_lineups()

Successfully received NBA Lineups from Rotowire


In [139]:
soup = get_beautiful_soup(roto_lineups_html)

In [140]:
games = list_of_games(soup)

In [183]:
nba_games = {}

In [185]:
nba_games = {}

game_data_list = []

for game in games:
    away_team_abbreviation_string = away_team_abbreviation(game)
    
    home_team_abbreviation_string = home_team_abbreviation(game)

    away_team_starters_list, away_team_lineup_status = away_team_starters(game)

    home_team_starters_list, home_team_lineup_status = home_team_starters(game)

    away_team_injury_list = away_team_injuries(game)

    home_team_injury_list = home_team_injuries(game)

    game_data = {"home_team_abbreviation" : home_team_abbreviation_string,
                 "home_team_starters": home_team_starters_list,
                 "home_team_injuries": home_team_injury_list,
                 "away_team_abbreviation" : away_team_abbreviation_string,
                 "away_team_starters": away_team_starters_list,
                 "away_team_injuries": away_team_injury_list}
    
    game_data_list.append(game_data)

    # print(away_team_abbreviation_string, home_team_abbreviation_string)
    # print(away_team_starters_list, away_team_lineup_status)
    # print(home_team_starters_list)
    # print(len(home_team_starters_list), home_team_lineup_status)
    # print(f"{away_team_abbreviation_string}: {away_team_injury_list}")

nba_games[todaysminute] = game_data_list

In [186]:
nba_games

{'202503031752': [{'home_team_abbreviation': 'PHI',
   'home_team_starters': [{'name': 'Tyrese Maxey',
     'position': 'PG',
     'status': 'Very Likely To Play',
     'status_abbr': 'healthy'},
    {'name': 'Quentin Grimes',
     'position': 'SG',
     'status': 'Very Likely To Play',
     'status_abbr': 'healthy'},
    {'name': 'Kelly Oubre',
     'position': 'SF',
     'status': 'Very Likely To Play',
     'status_abbr': 'healthy'},
    {'name': 'Paul George',
     'position': 'PF',
     'status': 'Toss Up To Play',
     'status_abbr': 'GTD'},
    {'name': 'Andre Drummond',
     'position': 'C',
     'status': 'Very Likely To Play',
     'status_abbr': 'healthy'}],
   'home_team_injuries': [{'name': 'Joel Embiid',
     'position': 'C',
     'status': 'Very Likely To Play',
     'status_abbr': 'OFS'},
    {'name': 'Eric Gordon',
     'position': 'G',
     'status': 'Very Likely To Play',
     'status_abbr': 'OFS'},
    {'name': 'Jared McCain',
     'position': 'G',
     'status': 'V

In [204]:
keys = list(nba_games.keys())

for game in nba_games[keys[0]]:
    print(game.keys())



dict_keys(['home_team_abbreviation', 'home_team_starters', 'home_team_injuries', 'away_team_abbreviation', 'away_team_starters', 'away_team_injuries'])
dict_keys(['home_team_abbreviation', 'home_team_starters', 'home_team_injuries', 'away_team_abbreviation', 'away_team_starters', 'away_team_injuries'])
dict_keys(['home_team_abbreviation', 'home_team_starters', 'home_team_injuries', 'away_team_abbreviation', 'away_team_starters', 'away_team_injuries'])
dict_keys(['home_team_abbreviation', 'home_team_starters', 'home_team_injuries', 'away_team_abbreviation', 'away_team_starters', 'away_team_injuries'])
dict_keys(['home_team_abbreviation', 'home_team_starters', 'home_team_injuries', 'away_team_abbreviation', 'away_team_starters', 'away_team_injuries'])
dict_keys(['home_team_abbreviation', 'home_team_starters', 'home_team_injuries', 'away_team_abbreviation', 'away_team_starters', 'away_team_injuries'])
dict_keys(['home_team_abbreviation', 'home_team_starters', 'home_team_injuries', 'away_t